In [59]:
import tensorflow as tf
import numpy as np

# Pre-requisites:
Please download the following files before you begin this tutorial:
- [balanced_train_segments.csv](http://storage.googleapis.com/us_audioset/youtube_corpus/v1/csv/balanced_train_segments.csv)
- [unbalanced_train_segments.csv](http://storage.googleapis.com/us_audioset/youtube_corpus/v1/csv/unbalanced_train_segments.csv)
- [eval_segments.csv](http://storage.googleapis.com/us_audioset/youtube_corpus/v1/csv/eval_segments.csv)
- [128-dimension audio features](https://research.google.com/audioset/download.html) i.e., embeddings - About 2GB in size.

`examples` must contain YouTube IDs of all examples for one class. Consider the class `Clapping`.

In [2]:
!grep Clapping class_labels_indices.csv

63,/m/0l15bq,"Clapping"


In [3]:
class_label_index = !grep Clapping class_labels_indices.csv

In [6]:
print class_label_index[0].split(",")[1]

/m/0l15bq


In [60]:
!grep /m/0l15bq balanced_train_segments.csv |head

0FMdORf5iGs, 30.000, 40.000, "/m/04rlf,/m/081rb,/m/09x0r,/m/0l15bq"
1IxBagCJeZc, 150.000, 160.000, "/m/01j3sz,/m/09x0r,/m/0l15bq"
1_DouJRW3PM, 30.000, 40.000, "/m/028ght,/m/09x0r,/m/0l15bq"
2y9ikTsTsl0, 30.000, 40.000, "/m/028ght,/m/09x0r,/m/0l15bq"
3PliaLqMSqg, 30.000, 40.000, "/m/028ght,/m/09x0r,/m/0l15bq"
3ixOXsKUufM, 30.000, 40.000, "/m/0l15bq"
4mOTOTJLv5U, 0.000, 10.000, "/m/09x0r,/m/0l15bq,/m/0ytgt"
7Ep2a7_sbmc, 260.000, 270.000, "/m/09x0r,/m/0l15bq"
7SpYywlGPyM, 30.000, 40.000, "/m/09x0r,/m/0k65p,/m/0l15bq,/m/0ytgt"
AiGF0850kT8, 6.000, 16.000, "/m/04rlf,/m/0l15bq"


In [61]:
examples = !grep /m/0l15bq balanced_train_segments.csv | head -2 | cut -c -11
print examples

['0FMdORf5iGs', '1IxBagCJeZc']


In [62]:
tfrecord_prefixes = set([i[:2] for i in examples])

In [63]:
tfrecord_filenames = ["bal_train/" + i + ".tfrecord" for i in tfrecord_prefixes]

In [64]:
audio_embeddings_dict = {}
audio_labels_dict = {}
#all_tfrecord_filenames = glob.glob("bal_train/" + example[:2] + ".tfrecord")

# Load embeddings
sess = tf.Session() 
for tfrecord in tfrecord_filenames: 
  for example in tf.python_io.tf_record_iterator(tfrecord):
    tf_example = tf.train.Example.FromString(example)
    vid_id = tf_example.features.feature['video_id'].bytes_list.value[0].decode(encoding='UTF-8')
    if vid_id in examples:
      example_label = list(np.asarray(tf_example.features.feature['labels'].int64_list.value))
      tf_seq_example = tf.train.SequenceExample.FromString(example)
      n_frames = len(tf_seq_example.feature_lists.feature_list['audio_embedding'].feature)
    
      audio_frame = []
      for i in range(n_frames):
        audio_frame.append(tf.cast(tf.decode_raw(
             tf_seq_example.feature_lists.feature_list['audio_embedding'].feature[i].bytes_list.value[0],tf.uint8)
            ,tf.float32).eval(session=sess))
      audio_embeddings_dict[vid_id] = audio_frame
      audio_labels_dict[vid_id] = example_label

In [65]:
print audio_labels_dict

{u'0FMdORf5iGs': [0, 63, 137, 387], u'1IxBagCJeZc': [0, 16, 63]}
